# *THIS IS TO CREATE THE DB*
You can download the already made DB file `financial_news.db` from the sharepoint 

[practicum folder](https://gtvault-my.sharepoint.com/:f:/g/personal/ltupac3_gatech_edu/Eg2gLDzQ8H1JoWUrUIq1G04BPkOXMyxmhgcoL84Q58-5dg?e=80dziH)

[db file](https://gtvault-my.sharepoint.com/:u:/g/personal/ltupac3_gatech_edu/Edi6YX6MKPxMud1e5maTIjsBo04ISTst1j7uoxeSVH2OBA?e=XQD3Ed)

In [1]:
import duckdb
import pandas as pd 
import os 
import xml.etree.ElementTree as ET

# Code to create database
Instructions:
1. For individual files (company_info_news.txt, volume_news.csv, etc.), copy the relative path to the respective variable below
2. For headline data, put the relative path to the folder housing the ticker folders.
    - Example: MultiCap_News/HEADLINES houses the individual ticker folders. 
    - The code will recursively pick up the files from there

In [2]:
con = duckdb.connect("financial_news.db")

company_txt_path = 'MultiCap_News\\company_info_news.txt'
volume_news_path = 'MultiCap_News\\volume_news.csv'
pricing_news_path = 'MultiCap_News\\pricing_news.csv'
multicap_headlines = 'MultiCap_News\\HEADLINES'
headline_august24_path = 'HEADLINES_August24'

In [3]:
# create the DDLs and indexes
ddl_statements = [
    "CREATE SCHEMA IF NOT EXISTS Headlines;",
    """
    CREATE TABLE IF NOT EXISTS headlines.Articles (
        guid TEXT,
        ticker TEXT,
        description TEXT,
        article_link TEXT,
        article_pubDate TIMESTAMP,
        article_title TEXT,
        language TEXT,
        lastBuildDate TIMESTAMP,
        link TEXT,
        title TEXT,
        PRIMARY KEY (guid, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Company_Info_News (
        ticker TEXT PRIMARY KEY,
        name TEXT,
        subindustry TEXT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Pricing_News (
        trading_day_date DATE,
        ticker TEXT,
        price FLOAT,
        PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Volume_News (
        trading_day_date DATE,
        ticker TEXT,
        volume INT,
        PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    ############ Gold Layer ############
    """
    CREATE TABLE IF NOT EXISTS headlines.Market_Data_Daily_Processing (
    trading_day_date DATE,
    ticker TEXT,
    price FLOAT,
    volume INT,
    PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Market_Data_Headlines (
    trading_day_date DATE,
    ticker TEXT,
    price FLOAT,
    volume INT,
    headline_count INT,
    PRIMARY KEY (trading_day_date, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Trading_Calendar (
    trading_date DATE PRIMARY KEY
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Articles_Trading_Day (
    guid TEXT,
    ticker TEXT,
    mapped_trading_date DATE,
    description TEXT,
    article_link TEXT,
    article_pubDate TIMESTAMP,
    article_title TEXT,
    language TEXT,
    lastBuildDate TIMESTAMP,
    link TEXT,
    title TEXT,
    PRIMARY KEY (guid, ticker)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS headlines.Market_Article_Summary (
    trading_date DATE PRIMARY KEY,
    article_count INT
    );
    """
]

drop_statements = [
    "DROP TABLE IF EXISTS headlines.Articles;",
    "DROP TABLE IF EXISTS headlines.Company_Info_News;",
    "DROP TABLE IF EXISTS headlines.Pricing_News;",
    "DROP TABLE IF EXISTS headlines.Volume_News;",
    "DROP TABLE IF EXISTS headlines.Market_Data_Daily_Processing;",
    "DROP TABLE IF EXISTS headlines.Market_Data_Headlines;",
    "DROP TABLE IF EXISTS headlines.Trading_Calendar;",
    "DROP TABLE IF EXISTS headlines.Articles_Trading_Day;",
    "DROP TABLE IF EXISTS headlines.Market_Article_Summary;"
]

index_statements = [
    "CREATE INDEX IF NOT EXISTS idx_articles_pubDate ON headlines.Articles (article_pubDate);",
    "CREATE INDEX IF NOT EXISTS idx_articles_pubDate ON headlines.Articles_Trading_Day (article_pubDate);"

]

for drop in drop_statements:
    con.execute(drop)

for ddl in ddl_statements:
    con.execute(ddl)

for index in index_statements:
    con.execute(index)


# Load Company Info News

In [4]:
# its all in one line
with open(company_txt_path, 'r') as file:
    lines = file.readline().split('\\n')
    # con.execute("TRUNCATE Company_Info_News")
    for line in lines[1:]:
        line = line.strip().split('|')
        # DONT RUN THIS TWICE BY MISTAKE!
        con.execute("INSERT INTO headlines.Company_Info_News VALUES (?,?,?)", line)

# Load `Volume_News` 

In [6]:
df = pd.read_csv(volume_news_path)
# df.head()

# convert the wide format to long format
volume_long_df = df.melt(id_vars=['Date'], var_name='Ticker', value_name='Volume')

# make sure they have the correct data types
volume_long_df['Date'] = pd.to_datetime(volume_long_df['Date'])
volume_long_df['Volume'] = pd.to_numeric(volume_long_df['Volume'], errors='coerce')

# con.execute("TRUNCATE Volume_News")
con.execute("INSERT INTO headlines.Volume_News (trading_day_date, ticker, Volume) SELECT Date, ticker, Volume FROM volume_long_df")

# Load `Pricing_News`

In [7]:
df = pd.read_csv(pricing_news_path)
# convert the wide format to long format
pricing_long_df = df.melt(id_vars=['Date'], var_name='Ticker', value_name='Price')

# make sure they have the correct data types
pricing_long_df['Date'] = pd.to_datetime(pricing_long_df['Date'])
pricing_long_df['Price'] = pd.to_numeric(pricing_long_df['Price'], errors='coerce')

# con.execute("TRUNCATE Pricing_News")
con.execute("INSERT INTO headlines.Pricing_News (trading_day_date, Ticker, price) SELECT Date, ticker, Price FROM pricing_long_df")

# Load `Market_Data_Daily_Processing`

In [8]:
# con.execute("TRUNCATE Market_Data_Daily_Processing")
con.execute("""
INSERT INTO headlines.Market_Data_Daily_Processing
SELECT 
    pn.trading_day_date,
    pn.ticker,
    pn.price,
    vn.volume
FROM 
    headlines.Pricing_News pn
LEFT JOIN 
    headlines.Volume_News vn 
ON 
    pn.trading_day_date = vn.trading_day_date AND pn.ticker = vn.ticker
""")

# Load `Trading_Calendar`

In [11]:
# pricing_dates = pricing_long_df['Date'].drop_duplicates()
# volume_dates = volume_long_df['Date'].drop_duplicates()

# trading_dates = pd.concat([pricing_dates, volume_dates]).drop_duplicates().sort_values()

# # make sure to have correct col name
# trading_dates_df = pd.DataFrame(trading_dates, columns=['trading_date'])
# con.execute("TRUNCATE Trading_Calendar")
con.execute(
"""
INSERT INTO headlines.Trading_Calendar
SELECT DISTINCT trading_day_date AS trading_date
FROM (
    SELECT trading_day_date FROM headlines.Pricing_News
    UNION
    SELECT trading_day_date FROM headlines.Volume_News
) AS all_dates
ORDER BY trading_date;
"""
)

# Load `Articles`

In [12]:
def xml_loader(base_dir):
    # lets do this in chunks instead
    failed_parses = pd.DataFrame()

    for root, dirs, files in os.walk(base_dir):
        data = [] 
        # extract ticker from foldername 
        ticker = os.path.basename(root)

        for file in files:
            file_path = os.path.join(root, file)
            
            try:
                tree = ET.parse(file_path)
                root_element = tree.getroot()
                
                channel = root_element.find('channel')
                if channel is not None:
                    # extract metadata info
                    language = channel.findtext("language") 
                    lastBuildDate = channel.findtext("lastBuildDate")
                    link = channel.findtext("link")
                    title = channel.findtext("title")
                    
                    # now meat and potatoes
                    for item in channel.findall("item"):
                        description = item.findtext("description")
                        guid = item.findtext("guid")
                        article_link = item.findtext("link")
                        article_pubDate = item.findtext("pubDate")
                        article_title = item.findtext("title")
                        
                        data.append({
                            "guid": guid,
                            "ticker": ticker,
                            "description": description,
                            "article_link": article_link,
                            "article_pubDate": article_pubDate,
                            "article_title": article_title,
                            "language": language,
                            "lastBuildDate": lastBuildDate,
                            "link": link,
                            "title": title
                        })
            except ET.parseError as e:
                print(f"Error parsing file {file_path}: {e}")
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
        
        # insert the data into the database
        if data:
            df = pd.DataFrame(data)
            # print("Performing timestamp coercion for", ticker)
            df['parsed_date'] = pd.to_datetime(df['article_pubDate'], errors='coerce')
            df['lastBuildDate'] = pd.to_datetime(df['lastBuildDate'], errors='coerce')
            # print("Done timestamp coercion for", ticker)
            
            # separate failed cases to avoid nulls
            current_failed = df[df['parsed_date'].isna()]
            current_valid = df[df['parsed_date'].notna()]

            # these are good
            current_valid = current_valid.assign(article_pubDate=current_valid['parsed_date']).drop(columns=['parsed_date'])

            # remove dupes on guid and ticker
            current_valid.drop_duplicates(subset=['guid', 'ticker'], inplace=True)

            
            failed_parses = pd.concat([failed_parses, current_failed], ignore_index=True)
            
            try:
                # adding this too just in case
                con.execute("INSERT INTO headlines.Articles SELECT * FROM current_valid ON CONFLICT (guid, ticker) DO NOTHING")
                print("inserted data for", ticker)
            except Exception as e:
                print(f"Error inserting data for {ticker}: {e}")
                
    failed_parses.to_csv("failed_article_dates.csv", index=False)
    return failed_parses

In [13]:
# GETTING NULLS! gonna fix the coercion logic
# con.execute("truncate Articles")

# load multicap headlines
failed_df = xml_loader(multicap_headlines)
# load new headlines
failed2_df = xml_loader(headline_august24_path)

inserted data for A
inserted data for AA
inserted data for AAL
inserted data for AAON
inserted data for AAP
inserted data for AAPL
inserted data for ABBV
inserted data for ABG
inserted data for ABNB
inserted data for ABT
inserted data for ACA
inserted data for ACAD
inserted data for ACGL
inserted data for ACHC
inserted data for ACI
inserted data for ACIW
inserted data for ACLS
inserted data for ACLX
inserted data for ACM
inserted data for ACN
inserted data for ACT
inserted data for ADBE
inserted data for ADC
inserted data for ADI
inserted data for ADM
inserted data for ADP
inserted data for ADSK
inserted data for ADT
inserted data for AEE
inserted data for AEIS
inserted data for AEL
inserted data for AEO
inserted data for AEP
inserted data for AER
inserted data for AES
inserted data for AFG
inserted data for AFL
inserted data for AFRM
inserted data for AGCO
inserted data for AGNC
inserted data for AGO
inserted data for AGR
inserted data for AI
inserted data for AIG
inserted data for AI

C:\Users\jovan\AppData\Local\Temp\ipykernel_20740\1589084342.py:54: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df['article_pubDate'], errors='coerce')


inserted data for Z
inserted data for ZBH
inserted data for ZBRA
inserted data for ZG
inserted data for ZI
inserted data for ZION
inserted data for ZM
inserted data for ZS
inserted data for ZTS
inserted data for ZWS
inserted data for A
inserted data for AA
inserted data for AAL
inserted data for AAON
inserted data for AAP
inserted data for AAPL
inserted data for ABBV
inserted data for ABCB
inserted data for ABG
inserted data for ABNB
inserted data for ABT
inserted data for ACA
inserted data for ACAD
inserted data for ACGL
inserted data for ACHC
inserted data for ACI
inserted data for ACIW
inserted data for ACLS
inserted data for ACLX
inserted data for ACM
inserted data for ACN
inserted data for ACT
inserted data for ADBE
inserted data for ADC
inserted data for ADI
inserted data for ADM
inserted data for ADMA
inserted data for ADP
inserted data for ADSK
inserted data for ADT
inserted data for AEE
inserted data for AEIS
inserted data for AEO
inserted data for AEP
inserted data for AER
in

In [14]:
failed_df.head()

,guid,ticker,description,article_link,article_pubDate,article_title,language,lastBuildDate,link,title,parsed_date
0,2d7d8468-a424-37c5-bd5a-b8bdb3574707,AAL,"NORTHAMPTON, MA / ACCESSWIRE / April 30, 2024 ...",https://finance.yahoo.com/news/ve-got-back-mee...,"Tue, 30 Apr 2024 14:45:00 +0000",Theyâve Got Your Back: Meet Americanâs Sys...,en-US,2024-05-04 13:00:58+00:00,http://finance.yahoo.com/q/h?s=AAL,Yahoo! Finance: AAL News,NaT
1,628e541a-a3ce-3f16-8e7d-10c27b5e76cd,AAL,Want AAdvantage Platinum status? It isn't abou...,https://www.fool.com/the-ascent/credit-cards/a...,"Sun, 28 Apr 2024 14:30:11 +0000",How Much Do You Need to Fly to Earn American A...,en-US,2024-05-04 13:00:58+00:00,http://finance.yahoo.com/q/h?s=AAL,Yahoo! Finance: AAL News,NaT
2,570c87a0-a8c8-331e-837f-b8477e62866b,AAL,"It's not how much you fly, it's how much you s...",https://www.fool.com/the-ascent/credit-cards/a...,"Sat, 27 Apr 2024 14:30:12 +0000",How Much Do You Need to Fly to Earn American A...,en-US,2024-05-04 13:00:58+00:00,http://finance.yahoo.com/q/h?s=AAL,Yahoo! Finance: AAL News,NaT
3,2b1b6b71-b2b5-3268-8cf8-f6b716e8d915,AAL,American Airlines Group Inc. ( NASDAQ:AAL ) la...,https://finance.yahoo.com/news/american-airlin...,"Sat, 27 Apr 2024 12:25:19 +0000",American Airlines Group Inc. (NASDAQ:AAL) Just...,en-US,2024-05-04 13:00:58+00:00,http://finance.yahoo.com/q/h?s=AAL,Yahoo! Finance: AAL News,NaT
4,76a935dc-e538-31d2-84c0-4b0d36787ad2,AAL,American Airlines Group Inc. (NASDAQ:AAL) Q1 2...,https://finance.yahoo.com/news/american-airlin...,"Sat, 27 Apr 2024 12:24:23 +0000",American Airlines Group Inc. (NASDAQ:AAL) Q1 2...,en-US,2024-05-04 13:00:58+00:00,http://finance.yahoo.com/q/h?s=AAL,Yahoo! Finance: AAL News,NaT


In [15]:
# try again with the faulty data 
failed_df['article_pubDate'] = pd.to_datetime(failed_df['article_pubDate'], errors='coerce')
failed_df = failed_df.drop(columns=['parsed_date'])
failed_df.drop_duplicates(subset=['guid', 'ticker'], inplace=True)
try:
    # adding this too just in case
    con.execute("INSERT INTO headlines.Articles SELECT * FROM failed_df ON CONFLICT (guid, ticker) DO NOTHING")
except Exception as e:
    print(f"Error inserting data: {e}")

In [16]:
# try again with the faulty data 
failed2_df['article_pubDate'] = pd.to_datetime(failed2_df['article_pubDate'], errors='coerce')
failed2_df = failed2_df.drop(columns=['parsed_date'])
failed2_df.drop_duplicates(subset=['guid', 'ticker'], inplace=True)
try:
    # adding this too just in case
    con.execute("INSERT INTO headlines.Articles SELECT * FROM failed2_df ON CONFLICT (guid, ticker) DO NOTHING")
except Exception as e:
    print(f"Error inserting data: {e}")

# Load `Articles_Trading_Day`

In [18]:
# con.execute("truncate Articles_Trading_Day")
con.execute("""
INSERT INTO headlines.Articles_Trading_Day
SELECT 
    a.guid,
    a.ticker,
    coalesce(MIN(tc.trading_date), cast(a.article_pubDate as Date)) AS mapped_trading_date,
    a.description,
    a.article_link,
    a.article_pubDate,
    a.article_title,
    a.language,
    a.lastBuildDate,
    a.link,
    a.title
FROM 
    headlines.Articles a
LEFT JOIN 
    headlines.Trading_Calendar tc
ON 
    tc.trading_date >= CAST(a.article_pubDate AS DATE) 
GROUP BY 
    a.guid, a.ticker, a.description, a.article_link, a.article_pubDate, 
    a.article_title, a.language, a.lastBuildDate, a.link, a.title;
""")

# Create `Market_Data_Headlines`

In [19]:
# con.execute("Truncate Market_Data_Headlines")
con.execute("""
INSERT INTO headlines.Market_Data_Headlines
SELECT 
    md.trading_day_date,
    md.ticker,
    md.price,
    md.volume,
    COALESCE(COUNT(DISTINCT atd.guid), 0) AS headline_count
FROM 
    headlines.Market_Data_Daily_Processing md
LEFT JOIN 
    headlines.Articles_Trading_Day atd
ON 
    md.ticker = atd.ticker AND md.trading_day_date = atd.mapped_trading_date
GROUP BY 
    md.trading_day_date, md.ticker, md.price, md.volume;
""")

# Load `market_article_summary`

In [20]:
# con.execute("drop table headlines.Market_Article_Summary")
# con.execute("""
#                 CREATE TABLE IF NOT EXISTS headlines.Market_Article_Summary (
#     trading_date DATE PRIMARY KEY,
#     article_count INT
#     );

#             """)
con.execute('''
INSERT INTO headlines.Market_Article_Summary
SELECT 
    atd.mapped_trading_date AS trading_date,
    COUNT(DISTINCT atd.guid) AS total_unique_articles
FROM 
    headlines.Articles_Trading_Day atd
GROUP BY 
    atd.mapped_trading_date;
''')

In [21]:
con.close()

# GOT S&P mapped, I'll do it later

In [22]:
con = duckdb.connect("financial_news.db")
sp500_volume_weekly_path = 'SP500\\volume.csv'
sp500_price_weekly_path = 'SP500\\price.csv'
sp500_price_daily_path = 'SP500\\price_daily.csv'
sp500_company_path = 'SP500\\company_info_sp500.txt'
sp500_price_sp500_path = 'SP500\\price_SP500.csv'
sp500_item1_path = 'SP500\\sp500_item1_sec_filings_0.txt'
sp500_item1a_path = 'SP500\\sp500_item1a_sec_filings_0.txt'
sp500_item7_path = 'SP500\\sp500_item7_sec_filings_0.txt'

In [23]:
# drop table statements 
drop_statements = [
    "DROP TABLE IF EXISTS SP500.Volume_Weekly;",
    "DROP TABLE IF EXISTS SP500.Price_Daily;",
    "DROP TABLE IF EXISTS SP500.Company_Info;",
    "DROP TABLE IF EXISTS SP500.Weekly_Market_Data;",
    "DROP TABLE IF EXISTS SP500.Price_Weekly;",
    "DROP TABLE IF EXISTS SP500.Price_Weekly_SP500;",
    "DROP TABLE IF EXISTS SP500.item7;",
    "DROP TABLE IF EXISTS SP500.item1a;",
    "DROP TABLE IF EXISTS SP500.item1;",
    "DROP TABLE IF EXISTS SP500.SEC_Item_Filings;"
]

ddl_statements = [
    "CREATE SCHEMA IF NOT EXISTS SP500;",
"""
CREATE TABLE IF NOT EXISTS SP500.Volume_Weekly (
    trading_week_date DATE NOT NULL,
    cik TEXT NOT NULL,
    volume FLOAT,
    PRIMARY KEY (trading_week_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Price_Daily (
    trading_day_date DATE NOT NULL,
    cik TEXT NOT NULL,
    price FLOAT,
    PRIMARY KEY (trading_day_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Company_Info (
    cik TEXT PRIMARY KEY,
    ticker TEXT,  
    name TEXT, 
    subindustry TEXT 
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Weekly_Market_Data (
    trading_week_date DATE NOT NULL,
    cik TEXT NOT NULL,
    price FLOAT NOT NULL,
    volume FLOAT NOT NULL,
    PRIMARY KEY (trading_week_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Price_Weekly (
    trading_week_date DATE NOT NULL,
    cik TEXT NOT NULL,
    price FLOAT,
    PRIMARY KEY (trading_week_date, cik)
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.Price_Weekly_SP500 (
    trading_week_date DATE PRIMARY KEY,
    SP500CapWeighted FLOAT,
    SP500EqualWeighted FLOAT
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.item7 (
    company TEXT NOT NULL,           
    filing_ts TIMESTAMP NOT NULL,              
    link TEXT,                       
    type TEXT,                       
    cik TEXT,                         
    item7 TEXT,                      
    PRIMARY KEY (company, filing_ts)      
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.item1a (
    company TEXT NOT NULL,           
    filing_ts TIMESTAMP NOT NULL,              
    link TEXT,                       
    type TEXT,                       
    cik TEXT,                         
    item1a TEXT,                     
    PRIMARY KEY (company, filing_ts)      
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.item1 (
    company TEXT NOT NULL,           
    filing_ts TIMESTAMP NOT NULL,              
    link TEXT,                       
    type TEXT,                       
    cik TEXT,                         
    item1 TEXT,                      
    PRIMARY KEY (company, filing_ts)      
);
""",
"""
CREATE TABLE IF NOT EXISTS SP500.SEC_Item_Filings (
    cik TEXT NOT NULL,               
    filing_ts TIMESTAMP NOT NULL,              
    item_filing TEXT NOT NULL,       
    company TEXT,                    
    link TEXT,                       
    type TEXT,                       
    item_description TEXT,           
    PRIMARY KEY (cik, filing_ts, item_filing)
);
"""
]

for drop in drop_statements:
    con.execute(drop)

for ddl in ddl_statements:
    con.execute(ddl)

# Load `volume_weekly`

In [24]:
df = pd.read_csv(sp500_volume_weekly_path)
# convert the wide format to long format
# volume_long_df = wide_to_long(df, ['Date'], 'Volume', 'cik')
volume_long_df = df.melt(id_vars=['Date'], var_name='cik', value_name='Volume')

# make sure they have the correct data types
volume_long_df['Date'] = pd.to_datetime(volume_long_df['Date'])
volume_long_df['Volume'] = pd.to_numeric(volume_long_df['Volume'], errors='coerce')

# default null volume values to 0. CIK 1534701 is all nulls so better to just drop the column but keep it for now
# nah jk leaving it as null for now. will coalesce the final table
# volume_long_df['Volume'] = volume_long_df['Volume'].fillna(0)

con.execute("INSERT INTO SP500.Volume_Weekly (trading_week_date, cik, volume) SELECT date, cik, volume FROM volume_long_df")

# Load `Price_Daily`

In [25]:
df = pd.read_csv(sp500_price_daily_path)
# convert the wide format to long format
# price_long_df = wide_to_long(df, ['Date'], 'Price', 'cik')
price_long_df = df.melt(id_vars=['Date'], var_name='cik', value_name='Price')

# make sure they have the correct data types
price_long_df['Date'] = pd.to_datetime(price_long_df['Date'])
price_long_df['Price'] = pd.to_numeric(price_long_df['Price'], errors='coerce')

# con.execute("TRUNCATE SP500.Price_Daily")
con.execute("INSERT INTO SP500.Price_Daily (trading_day_date, cik, price) SELECT Date, cik, Price FROM price_long_df")

In [37]:
price_long_df

,Date,cik,Price
0,2000-01-07,1534701,NaN
1,2000-01-14,1534701,NaN
2,2000-01-21,1534701,NaN
3,2000-01-28,1534701,NaN
4,2000-02-04,1534701,NaN
...,...,...,...
1122945,2023-03-10,814585,10.92
1122946,2023-03-17,814585,8.68
1122947,2023-03-24,814585,8.20
1122948,2023-03-31,814585,9.26


# Load `Price_Weekly`

<b>Looks like theres an issue with this dataset. When the office hours recordings come out I'll watch it and fix it. until then it'll be null </b>

In [26]:
df.head()

,Date,1534701,1341439,792985,1489393,86312,96289,1393612,40704,726513,...,75829,906345,84839,1122304,1526520,1013871,920760,814585,SP500CapWeighted,SP500EqualWeighted
0,2000-01-03,NaN,29.531250,14.5000,NaN,33.0000,54.0000,NaN,17.21875,52.500,...,20.5000,26.8125,0.888889,NaN,NaN,NaN,7.126582,34.458333,1455.219971,1414.1586
1,2000-01-04,NaN,26.921875,14.8125,NaN,32.5625,53.9375,NaN,16.90625,51.125,...,20.1875,26.1250,0.881573,NaN,NaN,NaN,7.070687,32.750000,1399.420044,1375.8559
2,2000-01-05,NaN,25.500000,15.6250,NaN,32.3125,56.3125,NaN,16.81250,50.375,...,19.5000,26.1875,0.877915,NaN,NaN,NaN,7.098634,33.041667,1402.109985,1382.7999
3,2000-01-06,NaN,24.000000,16.7500,NaN,32.9375,51.3750,NaN,16.78125,50.500,...,20.8125,26.6875,0.885231,NaN,NaN,NaN,7.098634,33.916667,1403.449951,1398.3869
4,2000-01-07,NaN,25.843750,17.8125,NaN,34.2500,50.9375,NaN,16.81250,50.500,...,21.0625,27.1875,0.892547,NaN,NaN,NaN,7.238371,34.250000,1441.469971,1429.7778


In [27]:
df = pd.read_csv(sp500_price_weekly_path)
# convert the wide format to long format
# price_long_df = wide_to_long(df, ['Date'], 'Price', 'cik')
price_long_df = df.melt(id_vars=['Date'], var_name='cik', value_name='Price')

# make sure they have the correct data types
price_long_df['Date'] = pd.to_datetime(price_long_df['Date'])
price_long_df['Price'] = pd.to_numeric(price_long_df['Price'], errors='coerce')

# con.execute("TRUNCATE SP500.Price_Weekly")
# con.execute("INSERT INTO SP500.Price_Weekly (trading_week_date, cik, price) SELECT Date, cik, price FROM price_long_df")

# Load `Company_Info`

In [28]:
# its all in one line
with open(sp500_company_path, 'r') as file:
    lines = file.readline().split('\\n')
    for line in lines[1:]:
        line = line.strip().split('|')
        # DONT RUN THIS TWICE BY MISTAKE!
        con.execute("INSERT INTO SP500.Company_Info VALUES (?,?,?,?)", line)

# Load `Weekly_Market_Data`

In [29]:
con.execute("""
INSERT INTO SP500.Weekly_Market_Data
SELECT 
    pw.trading_week_date AS trading_week_date,
    pw.cik AS cik,
    coalesce(pw.price, 0) AS price,
    coalesce(vw.volume, 0) AS volume
FROM 
    SP500.Price_Weekly pw
LEFT JOIN 
    SP500.Volume_Weekly vw
ON 
    pw.trading_week_date = vw.trading_week_date AND pw.cik = vw.cik
""")

# Load `SP500.Price_Weekly_SP500`

In [30]:
df = pd.read_csv(sp500_price_sp500_path)

df['Date'] = pd.to_datetime(df['Date'])
df['SP500CapWeighted'] = pd.to_numeric(df['SP500CapWeighted'], errors='coerce')
df['SP500EqualWeighted'] = pd.to_numeric(df['SP500EqualWeighted'], errors='coerce')

con.execute("truncate SP500.Price_Weekly_SP500")
con.execute("INSERT INTO SP500.Price_Weekly_SP500 (trading_week_date,SP500CapWeighted, SP500EqualWeighted) SELECT Date, SP500CapWeighted, SP500EqualWeighted FROM df")

# Load `item7 `

Sometimes the last columns comes in multiple lines. Sucks b/c its last column and I can't rely on the pipe, so gotta code for that 

In [31]:
# cols: company|date|link|type|cik|item7 
# con.execute("TRUNCATE SP500.item7")
def parse_items(file_path, table_name):
    # use 2 pointer approach to check if next line is a continuation of the current line
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        prev_line = None  

        for line in lines[1:]:
            line = line.strip()
            fields = line.split('|')

            if prev_line is None:
                prev_line = fields
                continue

            # if current line has required number of elements, insert prev line
            if len(fields) == 6:
                con.execute(f"INSERT INTO SP500.{table_name} VALUES (?,?,?,?,?,?)", prev_line)
                prev_line = fields
            else:
                prev_line[-1] += " " + line

        # the last record
        if prev_line:
            con.execute(f"INSERT INTO SP500.{table_name} VALUES (?,?,?,?,?,?)", prev_line)

In [32]:
con.execute("TRUNCATE SP500.item7")
parse_items(sp500_item7_path, 'item7')

# Load `item1`

In [33]:
con.execute("TRUNCATE SP500.item1")
parse_items(sp500_item1_path, 'item1')

# Load `item1a`

In [34]:
con.execute("TRUNCATE SP500.item1a")
parse_items(sp500_item1a_path, 'item1a')

# Load `SEC_Item_Filings`

I think the only thing different between the 3 item tables are the item filing # and the item description

With that said, I believe its better to use a longer table for simplicity vs a wider table 
- wide = instead of 1 item_filing and 1 item_description columns, we make a column for each filing and description

In [35]:
con.execute("""
INSERT INTO SP500.SEC_Item_Filings
SELECT 
    cik,
    filing_ts,
    '7' AS item_filing,
    company,
    link,
    type,
    item7 AS item_description
FROM 
    SP500.item7
UNION ALL
SELECT 
    cik,
    filing_ts,
    '1a' AS item_filing,
    company,
    link,
    type,
    item1a AS item_description
FROM 
    SP500.item1a
UNION ALL
SELECT 
    cik,
    filing_ts,
    '1' AS item_filing,
    company,
    link,
    type,
    item1 AS item_description
FROM 
    SP500.item1;
""")

In [36]:
con.close()